In [1]:
import pandas as pd
import plotly.express as px

from gtfparse import read_gtf

from datapaths import *

pd.set_option('display.max_rows', 4)
pd.set_option('display.max_columns', None)

In [2]:
flipon_to_data = pd.read_table(F_FLIPON_TO_DATA)
flipon_to_data


,Flipon,Coordinates,Strand,Gene Feature,Gene Name,Gene Strand,Gene Type,Gene ID,Transcript ID,c. M miR (+),c. M miR (-),c. MJ miR (+),c. MJ miR (-),c. J miR (+),c. J miR (-),all M miR (+),all M miR (-),all MJ miR (+),all MJ miR (-),all J miR (+),all J miR (-),cCRE (+-200bp),LINE/LTR (+-200bp),Feature Group
0,g4,chr1:3014794-3014871,-,Distal Intergenic,4933401J01Rik,+,TEC,ENSMUSG00000102693.1,ENSMUST00000193812.1,miR-328 (1),NaN,miR-486 (1),NaN,NaN,NaN,"miR-6994 (1), miR-129b (1)","miR-92a-2 (1), miR-698/miR-7078 (1), miR-3572 ...",miR-486a/miR-486b (1),"miR-3076 (1), miR-6418 (1), miR-6987 (1)",NaN,miR-7649 (1),NaN,"LINE@L1Md_F2(-), LINE@L1VL1(+)",LINE/LTR
1,g4,chr1:3099888-3099963,+,Promoter (2-3kb),Gm26206,+,snRNA,ENSMUSG00000064842.1,ENSMUST00000082908.1,NaN,miR-328 (1),NaN,NaN,NaN,NaN,"miR-365-1/miR-365-2 (1), miR-698/miR-7078 (1),...","miR-221 (1), miR-129b (1)","miR-6418 (1), miR-7067 (1)",NaN,NaN,NaN,NaN,"LINE@L1Md_F2(+), LTR@MTC(+)",LINE/LTR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77706,z-dna,chrY:4203100-4203146,.,Distal Intergenic,Gm28191,+,unprocessed_pseudogene,ENSMUSG00000100300.1,ENSMUST00000189112.1,NaN,NaN,miR-493 (1),NaN,NaN,NaN,"miR-466d/miR-466i (4), miR-466l (5)",NaN,miR-493 (1),NaN,NaN,NaN,NaN,LINE@L1_Mur1(-),LINE/LTR
77707,z-dna,chrY:4207093-4207113,.,Distal Intergenic,Gm28191,+,unprocessed_pseudogene,ENSMUSG00000100300.1,ENSMUST00000189112.1,NaN,NaN,NaN,NaN,NaN,NaN,"miR-466a/miR-466e/miR-466p (1), miR-466d/miR-4...",NaN,NaN,NaN,NaN,NaN,NaN,"LINE@L1_Mus3(+), LINE@L1Md_A(+), LTR@ORR1B1(+)",LINE/LTR


In [10]:
result = (
        flipon_to_data[flipon_to_data["Gene Feature"] == "Promoter (<=1kb)"]
        .groupby(["Gene ID", 'Gene Name', 'Gene Type'])["Flipon"]
        .value_counts()
        .rename("Count")
        .to_frame()
        .reset_index()
        .pivot_table(index=['Gene ID', 'Gene Name', 'Gene Type'], columns=['Flipon'], values=['Count']).fillna('')
        .droplevel(0, axis=1)
        .reset_index()
        .rename_axis(None,axis=1)
    )

result['Flipon types'] = (result['g4']!='').replace(True, 'G').replace(False,'') + (result['h-dna']!='').replace(True, 'H').replace(False,'') + (result['sidd']!='').replace(True, 'S').replace(False,'') + (result['z-dna']!='').replace(True, 'Z').replace(False,'')

result = result.sort_values('Gene Type').reset_index(drop=True)

result.to_csv(D_TABLES / "gene_list_with_pp_flipon_count.tsv", sep='\t', index=False)
result


,Gene ID,Gene Name,Gene Type,g4,h-dna,sidd,z-dna,Flipon types
0,ENSMUSG00000096150.1,Ighv1-85,IG_V_gene,,3.0,,,H
1,ENSMUSG00000076545.2,Igkv4-72,IG_V_gene,,,,1.0,Z
...,...,...,...,...,...,...,...,...
5712,ENSMUSG00000111554.1,Gm232,unprocessed_pseudogene,,,,1.0,Z
5713,ENSMUSG00000114410.1,Gm2048,unprocessed_pseudogene,1.0,,,,G
